In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from cmf import query

In [ ]:
crn = query(
    selector=select_crn,
    model=f"naive_{os.getenv('SCHEMA')}.crn",
    return_type="pandas",
    engine=db_engine[1],
)